In [1]:
# Essentials
import numpy as np
import pandas as pd
import datetime
import random

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

# Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.svm import SVR
from sklearn import neighbors
from mlxtend.regressor import StackingCVRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# Stats
from scipy import stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from math import sqrt

# Misc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
pd.set_option('display.max_columns', None)

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000

In [2]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import io
train = pd.read_csv('/content/drive/MyDrive/Shiv_Nadar_DAC/train.csv')
print(train.shape)
train.head()

(87864, 9)


,Item_ID,Item_W,Item_Type,Item_MRP,Outlet_ID,Outlet_Year,Outlet_Size,Outlet_Location_Type,Sales
0,FDU32,21.027499,Baking Goods,197.352319,OUT046,2004,Small,Tier 2,2689.457781
1,NCT54,21.102371,Meat,148.250214,OUT035,1987,Small,Tier 1,3437.350375
2,FDW08,20.882263,Hard Drinks,205.465010,OUT035,1999,Small,Tier 3,3129.967268
3,FDJ22,21.050435,Starchy Foods,253.417583,OUT046,1996,Small,Tier 1,1306.514376
4,FDF47,21.247876,Baking Goods,240.871039,OUT035,1988,Small,Tier 3,1739.769829


In [4]:
test = pd.read_csv('/content/drive/MyDrive/Shiv_Nadar_DAC/test.csv')
print(test.shape)
test.head()

(37656, 8)


,Item_ID,Item_W,Item_Type,Item_MRP,Outlet_ID,Outlet_Year,Outlet_Size,Outlet_Location_Type
0,DRM23,10.156725,Snack Foods,119.319482,OUT013,1999,High,Tier 3
1,FDG47,10.434425,Household,263.600449,OUT018,1987,High,Tier 3
2,FDN21,11.953589,Health and Hygiene,246.286990,OUT018,1997,High,Tier 1
3,FDZ23,14.948546,Dairy,136.200508,OUT046,2004,Medium,Tier 2
4,FDC16,11.080042,Canned,173.145664,OUT018,2009,Medium,Tier 3


In [5]:
sample = pd.read_csv('/content/drive/MyDrive/Shiv_Nadar_DAC/submission.csv')
print(sample.shape)
sample.head()

(37656, 1)


,Sales
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [6]:
train['OY'] = train['Outlet_Year'].astype(str)
test['OY'] = test['Outlet_Year'].astype(str)

In [7]:
def calc_smooth_mean(df1, df2, cat_name, target, weight):
    # Compute the global mean
    mean = stats.trim_mean(df1[target], 0.15)

    # Compute the number of values and the mean of each group
    agg = df1.groupby(cat_name)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + weight * mean) / (counts + weight)

    # Replace each value by the according smoothed mean
    if df2 is None:
        return df1[cat_name].map(smooth)
    else:
        return df1[cat_name].map(smooth),df2[cat_name].map(smooth.to_dict())

In [8]:
WEIGHT = 5
train['Item_ID_enc'], test['Item_ID_enc'] = calc_smooth_mean(df1=train, df2=test, cat_name='Item_ID', target='Sales', weight=WEIGHT)
train['Item_Type_enc'], test['Item_Type_enc'] = calc_smooth_mean(df1=train, df2=test, cat_name='Item_Type', target='Sales', weight=WEIGHT)
train['Outlet_ID_enc'], test['Outlet_ID_enc'] = calc_smooth_mean(df1=train, df2=test, cat_name='Outlet_ID', target='Sales', weight=WEIGHT)
train['OY_enc'], test['OY_enc'] = calc_smooth_mean(df1=train, df2=test, cat_name='OY', target='Sales', weight=WEIGHT)
train['Outlet_Size_enc'], test['Outlet_Size_enc'] = calc_smooth_mean(df1=train, df2=test, cat_name='Outlet_Size', target='Sales', weight=WEIGHT)
train['Outlet_Location_Type_enc'], test['Outlet_Location_Type_enc'] = calc_smooth_mean(df1=train, df2=test, cat_name='Outlet_Location_Type', target='Sales', weight=WEIGHT)

In [11]:
test['Item_ID_enc'] = test['Item_ID_enc'].fillna(test['Item_ID_enc'].mean())

In [12]:
target = "Sales"
train_target = train[target]
train.drop([target], axis=1, inplace=True)

In [13]:
alldata = pd.concat([train, test], axis=0, sort=False, ignore_index=True)
alldata.shape

(125520, 15)

In [14]:
alldata['median_Item_MRP_per_Item_W'] = alldata.groupby('Item_W')['Item_MRP'].transform('median')
alldata['min_Item_MRP_per_Item_W'] = alldata.groupby('Item_W')['Item_MRP'].transform('min')
alldata['max_Item_MRP_per_Item_W'] = alldata.groupby('Item_W')['Item_MRP'].transform('max')

alldata['median_Item_MRP_per_Item_ID'] = alldata.groupby('Item_ID')['Item_MRP'].transform('median')
alldata['min_Item_MRP_per_Item_ID'] = alldata.groupby('Item_ID')['Item_MRP'].transform('min')
alldata['max_Item_MRP_per_Item_ID'] = alldata.groupby('Item_ID')['Item_MRP'].transform('max')

alldata['median_Item_MRP_per_Item_Type'] = alldata.groupby('Item_Type')['Item_MRP'].transform('median')
alldata['min_Item_MRP_per_Item_Type'] = alldata.groupby('Item_Type')['Item_MRP'].transform('min')
alldata['max_Item_MRP_per_Item_Type'] = alldata.groupby('Item_Type')['Item_MRP'].transform('max')

alldata['median_Item_MRP_per_Outlet_ID'] = alldata.groupby('Outlet_ID')['Item_MRP'].transform('median')
alldata['min_Item_MRP_per_Outlet_ID'] = alldata.groupby('Outlet_ID')['Item_MRP'].transform('min')
alldata['max_Item_MRP_per_Outlet_ID'] = alldata.groupby('Outlet_ID')['Item_MRP'].transform('max')

alldata['median_Item_MRP_per_OY'] = alldata.groupby('OY')['Item_MRP'].transform('median')
alldata['min_Item_MRP_per_OY'] = alldata.groupby('OY')['Item_MRP'].transform('min')
alldata['max_Item_MRP_per_OY'] = alldata.groupby('OY')['Item_MRP'].transform('max')

alldata['median_Item_MRP_per_Outlet_Size'] = alldata.groupby('Outlet_Size')['Item_MRP'].transform('median')
alldata['min_Item_MRP_per_Outlet_Size'] = alldata.groupby('Outlet_Size')['Item_MRP'].transform('min')
alldata['max_Item_MRP_per_Outlet_Size'] = alldata.groupby('Outlet_Size')['Item_MRP'].transform('max')

alldata['median_Item_MRP_per_Outlet_Location_Type'] = alldata.groupby('Outlet_Location_Type')['Item_MRP'].transform('median')
alldata['min_Item_MRP_per_Outlet_Location_Type'] = alldata.groupby('Outlet_Location_Type')['Item_MRP'].transform('min')
alldata['max_Item_MRP_per_Outlet_Location_Type'] = alldata.groupby('Outlet_Location_Type')['Item_MRP'].transform('max')


alldata['median_Item_W_per_Item_ID'] = alldata.groupby('Item_ID')['Item_W'].transform('median')
alldata['min_Item_W_per_Item_ID'] = alldata.groupby('Item_ID')['Item_W'].transform('min')
alldata['max_Item_W_per_Item_ID'] = alldata.groupby('Item_ID')['Item_W'].transform('max')

alldata['median_Item_W_per_Item_Type'] = alldata.groupby('Item_Type')['Item_W'].transform('median')
alldata['min_Item_W_per_Item_Type'] = alldata.groupby('Item_Type')['Item_W'].transform('min')
alldata['max_Item_W_per_Item_Type'] = alldata.groupby('Item_Type')['Item_W'].transform('max')

alldata['median_Item_W_per_Outlet_ID'] = alldata.groupby('Outlet_ID')['Item_W'].transform('median')
alldata['min_Item_W_per_Outlet_ID'] = alldata.groupby('Outlet_ID')['Item_W'].transform('min')
alldata['max_Item_W_per_Outlet_ID'] = alldata.groupby('Outlet_ID')['Item_W'].transform('max')

alldata['median_Item_W_per_OY'] = alldata.groupby('OY')['Item_W'].transform('median')
alldata['min_Item_W_per_OY'] = alldata.groupby('OY')['Item_W'].transform('min')
alldata['max_Item_W_per_OY'] = alldata.groupby('OY')['Item_W'].transform('max')

alldata['median_Item_W_per_Outlet_Size'] = alldata.groupby('Outlet_Size')['Item_W'].transform('median')
alldata['min_Item_W_per_Outlet_Size'] = alldata.groupby('Outlet_Size')['Item_W'].transform('min')
alldata['max_Item_W_per_Outlet_Size'] = alldata.groupby('Outlet_Size')['Item_W'].transform('max')

alldata['median_Item_W_per_Outlet_Location_Type'] = alldata.groupby('Outlet_Location_Type')['Item_W'].transform('median')
alldata['min_Item_W_per_Outlet_Location_Type'] = alldata.groupby('Outlet_Location_Type')['Item_W'].transform('min')
alldata['max_Item_W_per_Outlet_Location_Type'] = alldata.groupby('Outlet_Location_Type')['Item_W'].transform('max')

In [15]:
train['Sales'] = train_target
alldata['median_Sales_per_Item_ID'] = alldata['Item_ID'].map(train.groupby('Item_ID')['Sales'].median().to_dict())
alldata['min_Sales_per_Item_ID'] = alldata['Item_ID'].map(train.groupby('Item_ID')['Sales'].min().to_dict())
alldata['max_Sales_per_Item_ID'] = alldata['Item_ID'].map(train.groupby('Item_ID')['Sales'].max().to_dict())

alldata['median_Sales_per_Item_Type'] = alldata['Item_Type'].map(train.groupby('Item_Type')['Sales'].median().to_dict())
alldata['min_Sales_per_Item_Type'] = alldata['Item_Type'].map(train.groupby('Item_Type')['Sales'].min().to_dict())
alldata['max_Sales_per_Item_Type'] = alldata['Item_Type'].map(train.groupby('Item_Type')['Sales'].max().to_dict())

alldata['median_Sales_per_Outlet_ID'] = alldata['Outlet_ID'].map(train.groupby('Outlet_ID')['Sales'].median().to_dict())
alldata['min_Sales_per_Outlet_ID'] = alldata['Outlet_ID'].map(train.groupby('Outlet_ID')['Sales'].min().to_dict())
alldata['max_Sales_per_Outlet_ID'] = alldata['Outlet_ID'].map(train.groupby('Outlet_ID')['Sales'].max().to_dict())

alldata['median_Sales_per_OY'] = alldata['OY'].map(train.groupby('OY')['Sales'].median().to_dict())
alldata['min_Sales_per_OY'] = alldata['OY'].map(train.groupby('OY')['Sales'].min().to_dict())
alldata['max_Sales_per_OY'] = alldata['OY'].map(train.groupby('OY')['Sales'].max().to_dict())

alldata['median_Sales_per_Outlet_Size'] = alldata['Outlet_Size'].map(train.groupby('Outlet_Size')['Sales'].median().to_dict())
alldata['min_Sales_per_Outlet_Size'] = alldata['Outlet_Size'].map(train.groupby('Outlet_Size')['Sales'].min().to_dict())
alldata['max_Sales_per_Outlet_Size'] = alldata['Outlet_Size'].map(train.groupby('Outlet_Size')['Sales'].max().to_dict())

alldata['median_Sales_per_Outlet_Location_Type'] = alldata['Outlet_Location_Type'].map(train.groupby('Outlet_Location_Type')['Sales'].median().to_dict())
alldata['min_Sales_per_Outlet_Location_Type'] = alldata['Outlet_Location_Type'].map(train.groupby('Outlet_Location_Type')['Sales'].min().to_dict())
alldata['max_Sales_per_Outlet_Location_Type'] = alldata['Outlet_Location_Type'].map(train.groupby('Outlet_Location_Type')['Sales'].max().to_dict())

In [16]:
alldata['total_years'] = 2021.0 - alldata['Outlet_Year']
alldata['Item_W_per_year'] = alldata.apply(lambda x: x['Item_W']/x['total_years'], axis = 1)
alldata['Item_MRP_per_year'] = alldata.apply(lambda x: x['Item_MRP']/x['total_years'], axis = 1)
alldata.drop('Outlet_Year', axis=1, inplace=True)

In [17]:
alldata['Item_ID_enc'] = alldata['Item_ID_enc'].fillna(alldata['Item_ID_enc'].mean())
alldata['median_Sales_per_Item_ID'] = alldata['median_Sales_per_Item_ID'].fillna(alldata['median_Sales_per_Item_ID'].mean())
alldata['min_Sales_per_Item_ID'] = alldata['min_Sales_per_Item_ID'].fillna(alldata['min_Sales_per_Item_ID'].mean())
alldata['max_Sales_per_Item_ID'] = alldata['max_Sales_per_Item_ID'].fillna(alldata['max_Sales_per_Item_ID'].mean())

In [18]:
ab = alldata[['Item_ID','Outlet_ID','Item_Type','OY','Outlet_Size','Outlet_Location_Type']]
ab.head()

,Item_ID,Outlet_ID,Item_Type,OY,Outlet_Size,Outlet_Location_Type
0,FDU32,OUT046,Baking Goods,2004,Small,Tier 2
1,NCT54,OUT035,Meat,1987,Small,Tier 1
2,FDW08,OUT035,Hard Drinks,1999,Small,Tier 3
3,FDJ22,OUT046,Starchy Foods,1996,Small,Tier 1
4,FDF47,OUT035,Baking Goods,1988,Small,Tier 3


In [19]:
features = pd.get_dummies(ab, columns=ab.columns, sparse=True)
features = features.sparse.to_coo()

In [20]:
num_cols = [col for col in alldata.columns if col not in ab.columns]

In [21]:
import scipy
num_features=scipy.sparse.coo_matrix(alldata[num_cols].values)

In [22]:
features=scipy.sparse.hstack([features, num_features]).tocsr()

In [23]:
train_ohe = features[:train.shape[0], :]
test_ohe = features[train.shape[0]:, :]

print(train_ohe.shape)
print(test_ohe.shape)

(87864, 1003)
(37656, 1003)


In [24]:
# Setup cross validation folds
kf = KFold(n_splits=4, random_state=42, shuffle=True)

In [25]:
#Define error metrics
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model):
    rmse = np.sqrt(-cross_val_score(model, train_ohe, train_target, scoring="neg_mean_squared_error", cv=kf))
    return (rmse)

In [26]:
lightgbm = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=1000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

# ridge regressor
ridge = Ridge(alpha=0.35)
# lasso regressor
lasso = Lasso(alpha=.5)
# elasticnet regressor
elasticnet = ElasticNet(alpha=1.0, l1_ratio=0.5)

# Stack up all the models above, optimized using rf
stack_gen = StackingCVRegressor(regressors=(lightgbm ,ridge, lasso, elasticnet),
                                meta_regressor=elasticnet,
                                use_features_in_secondary=True)

In [28]:
scores = {}
score = cv_rmse(lightgbm)
print("lightgbm: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['lgb'] = (score.mean(), score.std())

lightgbm: 1267.2222 (5.7255)


In [29]:
score = cv_rmse(ridge)
print("ridge: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['ridge'] = (score.mean(), score.std())

ridge: 1265.3948 (5.3033)


In [30]:
score = cv_rmse(lasso)
print("lasso: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['lasso'] = (score.mean(), score.std())

lasso: 1265.1988 (5.3601)


In [31]:
score = cv_rmse(elasticnet)
print("elasticnet: {:.4f} ({:.4f})".format(score.mean(), score.std()))
scores['elasticnet'] = (score.mean(), score.std())

elasticnet: 1265.1802 (5.3095)


In [32]:
print('stack_gen')
stack_gen_model = stack_gen.fit(train_ohe, train_target)

stack_gen


In [33]:
print('lightgbm')
lgb_model_full_data = lightgbm.fit(train_ohe, train_target)

lightgbm


In [34]:
print('ridge')
ridge_model_full_data = ridge.fit(train_ohe, train_target)

ridge


In [35]:
print('lasso')
lasso_model_full_data = lasso.fit(train_ohe, train_target)

lasso


In [36]:
print('elasticnet')
elasticnet_model_full_data = elasticnet.fit(train_ohe, train_target)

elasticnet


In [37]:
# Blend models in order to make the final predictions more robust to overfitting
def blended_predictions(train_ohe):
    return (((0.10 * lgb_model_full_data.predict(train_ohe)) + \
            (0.20 * ridge_model_full_data.predict(train_ohe)) + \
            (0.20 * lasso_model_full_data.predict(train_ohe)) + \
            (0.20 * elasticnet_model_full_data.predict(train_ohe)) + \
            (0.30 * stack_gen_model.predict(train_ohe))))

In [38]:
# Get final precitions from the blended model
blended_score = rmsle(train_target, blended_predictions(train_ohe))
scores['blended'] = (blended_score, 0)
print('RMSLE score on train data:')
print(blended_score)

RMSLE score on train data:
1263.5894425686186


In [39]:
pred = blended_predictions(test_ohe)
sample['Sales'] = pred

In [41]:
from google.colab import files

sample.to_csv('subP.csv', index=False)
files.download('subP.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>